In [ ]:
import numpy as np
import pandas as pd

In [ ]:
'''
  id - The image ID.
  organ - The organ that the biopsy sample was taken from.
  data_source - Whether the image was provided by HuBMAP or HPA.
  img_height - The height of the image in pixels.
  img_width - The width of the image in pixels.
  pixel_size - The height/width of a single pixel from this image in micrometers. 
    All HPA images have a pixel size of 0.4 µm. For HuBMAP imagery the pixel size is 0.5 µm for kidney,
    0.2290 µm for large intestine, 0.7562 µm for lung, 0.4945    for spleen, and 6.263 µm for prostate.
  tissue_thickness - The thickness of the biopsy sample in micrometers. All HPA images have a thickness of 4 µm.
    The HuBMAP samples have tissue slice thicknesses 10 µm for kidney, 8 µm for large intestine, 4 µm for spleen,
    5 µm for lung, and 5 µm for prostate.
  rle - The target column. A run length encoded copy of the annotations. Provided for the training set only.
  age - The patient's age in years. Provided for the training set only.
  sex - The sex of the patient. Provided for the training set only.
'''
train = pd.read_csv('./train.csv', encoding='utf8')

# Descripcion baga de los datos

In [ ]:
train.describe()

In [ ]:
__quant = list(train.describe().columns)

for variable in train.columns:
  if (variable in __quant or variable == 'rle'): continue
  print(train[variable].unique())


# Analisis Exploratorio